In [71]:
# conjugate gradient
function conjugate_gradient(A, b; x0=zero(b), tol=1e-8, maxiter=1000)
    x = copy(x0) # current estimate
    r = b - A * x # residual: how far x is from the true solution
    p = copy(r) # search direction
    rs_old = dot(r, r) # squared norm of the residual

    residuals = [sqrt(rs_old)] # store the first residual

    for k in 1:maxiter
        Ap = A * p  # determine the best step size
        alpha = rs_old / dot(p, Ap) # optimal step length along direction p
        x += alpha * p # move in direction p by amount α
        r -= alpha * Ap # update residual

        # Check Convergence: If residual norm is small enough, CG has converged
        rs_new = dot(r, r)
        push!(residuals, sqrt(rs_new))
        if sqrt(rs_new) < tol
            println("Converged in $k iterations")
            return x
        end

        beta = rs_new / rs_old  # how much of the previous direction is kept

        # Update Search Direction
        p = r + beta * p
        rs_old = rs_new
    end
    # incase max iterations are reached
    println("Max iterations reached")
    return x, residuals
end


conjugate_gradient (generic function with 1 method)

In [96]:
using Random, LinearAlgebra

# Fix the seed
Random.seed!(123)  # <- same seed every time

# Define your SPD matrices and vector b
n = 5
A1 = spd_matrix(n, 10)
A2 = spd_matrix(n, 1000)
A3 = spd_matrix(n, 1000000000)
b = randn(n)  # this will always be the same with the same seed

# Run conjugate gradient
x1, res1 = conjugate_gradient(A1, b)
x2, res2 = conjugate_gradient(A2, b)
x3, res3 = conjugate_gradient(A3, b)


Converged in 5 iterations
Converged in 5 iterations
Converged in 8 iterations


5-element Vector{Float64}:
 -0.2741599129813721
  0.2991535128301981
 -0.24148689541506838
 -0.22763826366089504
  0.18287196149967042

In [80]:
using Plots

x1, res1 = conjugate_gradient_residuals(A1, b)
x2, res2 = conjugate_gradient_residuals(A2, b)
x3, res3 = conjugate_gradient_residuals(A3, b)

plt = plot(1:length(res1), res1, yscale=:log10, label="cond=10",
           xlabel="Iteration", ylabel="Residual ||r||")
plot!(plt, 1:length(res2), res2, label="cond=1000")
plot!(plt, 1:length(res3), res3, label="cond=10000000")

# Save the plot as a PNG (or PDF, SVG, etc.)
savefig(plt, "cg_residuals.png")

Converged in 5 iterations
Converged in 5 iterations
Converged in 6 iterations


"/content/cg_residuals.png"

In [81]:
using LinearAlgebra

function gmres_residuals(A, b; x0=zero(b), tol=1e-8, maxiter=100)
    n = length(b)
    x = copy(x0)
    r0 = b - A*x
    beta = norm(r0)

    # Initialize residual storage
    residuals = [beta]

    if beta < tol
        return x, residuals
    end

    # Arnoldi process
    V = zeros(n, maxiter+1)
    H = zeros(maxiter+1, maxiter)
    V[:,1] = r0 / beta

    for j in 1:maxiter
        w = A * V[:,j]
        for i in 1:j
            H[i,j] = dot(V[:,i], w)
            w -= H[i,j] * V[:,i]
        end
        H[j+1,j] = norm(w)
        if H[j+1,j] != 0
            V[:,j+1] = w / H[j+1,j]
        end

        # Solve least squares min ||beta e1 - H y||
        e1 = zeros(j+1); e1[1] = beta
        y = H[1:j+1,1:j] \ e1
        x_approx = x0 + V[:,1:j]*y
        res = norm(b - A*x_approx)
        push!(residuals, res)

        if res < tol
            println("GMRES converged in $j iterations")
            return x_approx, residuals
        end
    end
    println("GMRES max iterations reached")
    return x_approx, residuals
end


gmres_residuals (generic function with 1 method)

In [97]:
using Random

n = 5
Random.seed!(123)

A1 = spd_matrix(n, 10)    # well-conditioned
A2 = spd_matrix(n, 1000)  # ill-conditioned
A3 = spd_matrix(n, 1000000000)  # ill-conditioned
b = randn(n)

x1, res1 = gmres_residuals(A1, b)
x2, res2 = gmres_residuals(A2, b)
x3, res3 = gmres_residuals(A3, b)



GMRES converged in 5 iterations
GMRES converged in 5 iterations
GMRES converged in 20 iterations


([-0.27415991495884606, 0.2991535149879471, -0.24148689715687685, -0.22763826530281545, 0.18287196281869827], [1.70277456031669, 0.5912551020596135, 0.5587984966545378, 0.5551925664698146, 0.5551879967344188, 5.3208493770917836e-8, 6.608632795677502e-8, 7.320522511243822e-8, 5.53699677308861e-8, 9.680904872554922e-8  …  4.4586443967389273e-8, 1.0293551177795117e-7, 9.867466675787118e-8, 8.22991103316705e-8, 7.121711277261555e-8, 9.383161776885971e-8, 3.872988108191762e-8, 6.366484589815538e-8, 2.5988694872809017e-8, 8.931817266446315e-9])

In [83]:
using Plots


x1, res1 = gmres_residuals(A1, b)
x2, res2 = gmres_residuals(A2, b)
x3, res3 = gmres_residuals(A3, b)

plt = plot(1:length(res1), res1, yscale=:log10, label="cond=10", xlabel="Iteration", ylabel="Residual ||r||")
plot!(plt, 1:length(res2), res2, label="cond=1000")  # yscale log10 already set
plot!(plt, 1:length(res3), res3, label="cond=10000000")  # yscale log10 already set

savefig(plt, "gmres_residuals.png")


GMRES converged in 5 iterations
GMRES converged in 5 iterations
GMRES converged in 5 iterations


"/content/gmres_residuals.png"

In [84]:
using LinearAlgebra

function bicgstab_residuals(A, b; x0=zero(b), tol=1e-8, maxiter=1000)
    n = length(b)
    x = copy(x0)
    r = b - A*x
    r_hat = copy(r)
    rho_old = alpha = omega = 1.0
    v = zeros(n)
    p = zeros(n)

    residuals = [norm(r)]

    for k in 1:maxiter
        rho = dot(r_hat, r)
        if rho == 0
            println("Breakdown: rho=0")
            break
        end
        if k == 1
            p = r
        else
            beta = (rho/rho_old) * (alpha/omega)
            p = r + beta*(p - omega*v)
        end

        v = A*p
        alpha = rho / dot(r_hat, v)
        s = r - alpha*v
        t = A*s
        omega = dot(t,s)/dot(t,t)
        x += alpha*p + omega*s
        r = s - omega*t

        push!(residuals, norm(r))

        if norm(r) < tol
            println("BiCGSTAB converged in $k iterations")
            return x, residuals
        end

        if omega == 0
            println("Breakdown: omega=0")
            break
        end

        rho_old = rho
    end
    println("BiCGSTAB max iterations reached")
    return x, residuals
end


bicgstab_residuals (generic function with 1 method)

In [99]:
Random.seed!(123)

A1 = spd_matrix(n, 10)    # well-conditioned
A2 = spd_matrix(n, 1000)  # ill-conditioned
A3 = spd_matrix(n, 1000000000)
b = randn(n)

x1, res1 = bicgstab_residuals(A1, b)
x2, res2 = bicgstab_residuals(A2, b)
x3, res3 = bicgstab_residuals(A3, b)


BiCGSTAB converged in 5 iterations
BiCGSTAB converged in 5 iterations
BiCGSTAB converged in 7 iterations


([-0.27415991539548556, 0.29915351546439256, -0.2414868975414798, -0.2276382656653624, 0.1828719631099485], [1.70277456031669, 0.5588090033879024, 0.5774231804324065, 0.5774926907556334, 0.5763974370786262, 5.405345939287912e-8, 1.0150579982080802e-8, 1.7742344876623174e-9])

In [86]:
using Plots

x1, res1 = bicgstab_residuals(A1, b)
x2, res2 = bicgstab_residuals(A2, b)
x3, res3 = bicgstab_residuals(A3, b)

plt = plot(1:length(res1), res1, yscale=:log10, label="cond=10", xlabel="Iteration", ylabel="Residual ||r||")
plot!(plt, 1:length(res2), res2, label="cond=1000")  # yscale log10 already set
plot!(plt, 1:length(res3), res3, label="cond=10000000")  # yscale log10 already set

savefig(plt, "bicgstab_residuals.png")


BiCGSTAB converged in 5 iterations
BiCGSTAB converged in 5 iterations
BiCGSTAB converged in 5 iterations


"/content/bicgstab_residuals.png"

In [87]:
# CG
x1_cg, res1_cg = conjugate_gradient_residuals(A1, b)
x2_cg, res2_cg = conjugate_gradient_residuals(A2, b)
x3_cg, res3_cg = conjugate_gradient_residuals(A3, b)

# GMRES
x1_gm, res1_gm = gmres_residuals(A1, b)
x2_gm, res2_gm = gmres_residuals(A2, b)
x3_gm, res3_gm = gmres_residuals(A3, b)

# BiCGSTAB
x1_bi, res1_bi = bicgstab_residuals(A1, b)
x2_bi, res2_bi = bicgstab_residuals(A2, b)
x3_bi, res3_bi = bicgstab_residuals(A3, b)

# Extract final residuals
final_res_cg_10    = res1_cg[end]
final_res_cg_1000  = res2_cg[end]
final_res_cg_100000  = res3_cg[end]
println(final_res_cg_10)
println(final_res_cg_1000)
println(final_res_cg_100000)

final_res_gm_10    = res1_gm[end]
final_res_gm_1000  = res2_gm[end]
final_res_gm_100000  = res3_gm[end]
println(final_res_gm_10)
println(final_res_gm_1000)
println(final_res_gm_100000)

final_res_bi_10    = res1_bi[end]
final_res_bi_1000  = res2_bi[end]
final_res_bi_100000  = res3_bi[end]
println(final_res_bi_10)
println(final_res_bi_1000)
println(final_res_bi_100000)


Converged in 5 iterations
Converged in 5 iterations
Converged in 6 iterations
GMRES converged in 5 iterations
GMRES converged in 5 iterations
GMRES converged in 5 iterations
BiCGSTAB converged in 5 iterations
BiCGSTAB converged in 5 iterations
BiCGSTAB converged in 5 iterations
2.4276833723487815e-16
4.0181267127755384e-14
1.9565518504412037e-9
3.0182194481103634e-16
1.7278826712826995e-14
1.1603178684643001e-9
7.588418996682605e-18
1.8865675115385954e-15
2.9083651242161397e-9


In [88]:
t_cg_10 = @elapsed x1_cg, res1_cg = conjugate_gradient_residuals(A1, b)
t_cg_1000 = @elapsed x2_cg, res2_cg = conjugate_gradient_residuals(A2, b)
t_cg_100000 = @elapsed x3_cg, res3_cg = conjugate_gradient_residuals(A3, b)

t_gm_10 = @elapsed x1_gm, res1_gm = gmres_residuals(A1, b)
t_gm_1000 = @elapsed x2_gm, res2_gm = gmres_residuals(A2, b)
t_gm_100000 = @elapsed x3_gm, res3_gm = gmres_residuals(A3, b)

t_bi_10 = @elapsed x1_bi, res1_bi = bicgstab_residuals(A1, b)
t_bi_1000 = @elapsed x2_bi, res2_bi = bicgstab_residuals(A2, b)
t_bi_100000 = @elapsed x3_bi, res3_bi = bicgstab_residuals(A3, b)


Converged in 5 iterations
Converged in 5 iterations
Converged in 6 iterations
GMRES converged in 5 iterations
GMRES converged in 5 iterations
GMRES converged in 5 iterations
BiCGSTAB converged in 5 iterations
BiCGSTAB converged in 5 iterations
BiCGSTAB converged in 5 iterations


0.000136422

In [89]:
println("CG time (cond=10): ", t_cg_10)
println("CG time (cond=1000): ", t_cg_1000)
println("CG time (cond=100000): ", t_cg_100000)

println("GMRES time (cond=10): ", t_gm_10)
println("GMRES time (cond=1000): ", t_gm_1000)
println("GMRES time (cond=100000): ", t_gm_100000)

println("BiCGSTAB time (cond=10): ", t_bi_10)
println("BiCGSTAB time (cond=1000): ", t_bi_1000)
println("BiCGSTAB time (cond=100000): ", t_bi_100000)


CG time (cond=10): 0.00014233
CG time (cond=1000): 0.001170614
CG time (cond=100000): 0.000123069
GMRES time (cond=10): 0.000244225
GMRES time (cond=1000): 0.000241198
GMRES time (cond=100000): 0.000258137
BiCGSTAB time (cond=10): 0.00025974
BiCGSTAB time (cond=1000): 0.000124373
BiCGSTAB time (cond=100000): 0.000136422
